# Backups
# Get session Info
- Using API key make a request get ascociated `session_info` 
- `session_info` contains current user and current customer uuid.

In [5]:
import http.client
import json
import os
from pprint import pprint

platform_address = os.getenv('API_BASE_URL')
platform_api_key = os.getenv('YB_API_KEY')

conn = http.client.HTTPConnection(f"{platform_address}")

headers = {
  'Content-Type': "application/json",
  'X-AUTH-YW-API-TOKEN': f"{platform_api_key}"
}

conn.request("GET", "/api/v1/session_info", headers=headers)

res = conn.getresponse()
data = res.read()
session_info = json.loads(data)
pprint(session_info)

{'apiToken': '2c76dd83-baeb-460c-b7ef-cb5ba516ed37',
 'customerUUID': '11d78d93-1381-4d1d-8393-ba76f47ba7a6',
 'userUUID': '52bce00c-dd1b-dcd7-0821-b8070a7173c5'}


#### Storage Configuration Parameter
- Now pick a storage config you want to use to store the backups.
See `./list-customer-config.ipynb` for how to list all configs

In [6]:
storage_config_uuid = os.getenv('STORAGE_CONF_UUID',
                                "f5cfe9fb-cd4f-46e7-8ed8-af285b730cc0")

#### Universe UUID Parameter
uuid of universe to be backed up. See `list-universes.ipynb` for how to

In [7]:
universe_uuid = os.getenv('UNIVERSE_UUID',
                          "5e16b61c-13d8-4141-8d5d-edbd52a3489e")

### Create Scheduled Whole Universe Backup
- schedule frequency (i.e. take backup every) 1 day (86400000 ms) 
- delete (garbage collect) backup after 10 days (864000000 ms)

In [ ]:

payload = ('{\n'
           '  "actionType": "CREATE",\n'
           '  "keyspace": "postgres",\n'
           f'  "storageConfigUUID": "{storage_config_uuid}",\n'
           '  "timeBeforeDelete": "864000000",\n'
           '  "sse": "false",\n'
           '  "transactionalBackup": "false",\n'
           '  "parallelism": "8",\n'
           '  "schedulingFrequency": "86400000",\n'
           '  "backupType": "PGSQL_TABLE_TYPE"\n'
           '}')

url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/universes/{universe_uuid}/multi_table_backup"
conn.request("PUT", url, payload, headers)

res = conn.getresponse()
schedule_info = json.load(res)
pprint(schedule_info)

### Delete backup Schedule

In [ ]:
url = f"/api/v1/customers/{session_info.get('customerUUID')}" \
      f"/schedules/{schedule_info.get("scheduleUUID")}"
conn.request("DELETE", url, headers=headers)